In [7]:
import pandas as pd
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.ensemble import RandomForestRegressor
import numpy as np

# train.csv 파일 불러오기
train_data = pd.read_csv('train.csv')

# 결측치를 평균값으로 대체
train_data = train_data.fillna(train_data.mean())

# '측정 시간대'를 원핫 인코딩하여 숫자 형태로 변환
encoder = OneHotEncoder(sparse=False)
time_encoded = encoder.fit_transform(train_data[['측정 시간대']])
time_encoded_df = pd.DataFrame(time_encoded, columns=encoder.get_feature_names(['측정 시간대']))
train_data = pd.concat([train_data, time_encoded_df], axis=1).drop(['측정 시간대'], axis=1)

# 풍속을 예측할 특성(입력 변수)과 풍속(출력 변수)을 분리합니다.
X_train = train_data.drop(['ID', '풍속 (m/s)'], axis=1)  # 입력 변수들
y_train = train_data['풍속 (m/s)']  # 출력 변수 (풍속)

# HyperOpt를 사용하여 최적의 랜덤 포레스트 모델 찾기
def objective(params):
    model = RandomForestRegressor(**params, random_state=42)
    score = -np.mean(cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_absolute_error'))
    return {'loss': score, 'status': STATUS_OK}

space = {
    'n_estimators': hp.quniform('n_estimators', 50, 200, 1),
    'max_depth': hp.quniform('max_depth', 5, 30, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 10, 1),
    'min_samples_leaf': hp.quniform('min_samples_leaf', 1, 10, 1),
}

trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=100, trials=trials)

# 최적의 하이퍼파라미터로 모델 생성
best_model = RandomForestRegressor(**best, random_state=42)
best_model.fit(X_train, y_train)

# test.csv 파일 불러오기
test_data = pd.read_csv('test.csv')

# 결측치를 평균값으로 대체
test_data = test_data.fillna(test_data.mean())

# '측정 시간대'를 원핫 인코딩하여 숫자 형태로 변환
time_encoded = encoder.transform(test_data[['측정 시간대']])
time_encoded_df = pd.DataFrame(time_encoded, columns=encoder.get_feature_names(['측정 시간대']))
test_data = pd.concat([test_data, time_encoded_df], axis=1).drop(['측정 시간대'], axis=1)

# 테스트 데이터로 풍속 예측을 수행합니다.
X_test = test_data.drop('ID', axis=1)  # 테스트 입력 변수들
y_pred = best_model.predict(X_test)  # 테스트 데이터로 풍속 예측

# Submit / 제출
submission = pd.read_csv('./sample_submission.csv')
submission['풍속 (m/s)'] = y_pred

# 예측 결과를 submission.csv 양식에 맞게 저장합니다.
submission.to_csv('submission.csv', index=False)

print("풍속 예측이 완료되었습니다. 결과가 submission.csv에 저장되었습니다.")


  2%|█▎                                                                | 2/100 [00:00<00:05, 18.74trial/s, best loss=?]

C:\Users\user\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  del sys.path[0]
C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_validation.py:614: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\ensemble\_forest.py", line 349, in fit
    self._validate_estimator()
  File "C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\ensemble\_base.py", line 130, in _validate_estimator
 

  7%|████▌                                                             | 7/100 [00:00<00:04, 20.09trial/s, best loss=?]

C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_validation.py:614: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\ensemble\_forest.py", line 349, in fit
    self._validate_estimator()
  File "C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\ensemble\_base.py", line 130, in _validate_estimator
    "got {0}.".format(type(self.n_estimators)))
ValueError: n_estimators must be an integer, got <class 'float'>.

  FitFailedWarning)

C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_validation.py:614: FitFailedWarning: Estimator fit failed. The score on this train-test partition 

 10%|██████▌                                                          | 10/100 [00:00<00:04, 20.82trial/s, best loss=?]

...

 97%|███████████████████████████████████████████████████████████████  | 97/100 [00:04<00:00, 25.53trial/s, best loss=?]

C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_validation.py:614: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\ensemble\_forest.py", line 349, in fit
    self._validate_estimator()
  File "C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\ensemble\_base.py", line 130, in _validate_estimator
    "got {0}.".format(type(self.n_estimators)))
ValueError: n_estimators must be an integer, got <class 'float'>.

  FitFailedWarning)

C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_validation.py:614: FitFailedWarning: Estimator fit failed. The score on this train-test partition 

100%|████████████████████████████████████████████████████████████████| 100/100 [00:04<00:00, 23.94trial/s, best loss=?]


C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_validation.py:614: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\ensemble\_forest.py", line 349, in fit
    self._validate_estimator()
  File "C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\ensemble\_base.py", line 130, in _validate_estimator
    "got {0}.".format(type(self.n_estimators)))
ValueError: n_estimators must be an integer, got <class 'float'>.

  FitFailedWarning)

C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_validation.py:614: FitFailedWarning: Estimator fit failed. The score on this train-test partition 

AllTrialsFailed: 